# IMPORTY

In [ ]:
%%capture --no-stderr

import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from datetime import datetime
from pandas.api.types import is_numeric_dtype
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import math
np.random.seed(29)

# Preprocess

In [ ]:
# Wczytanie Danych
df = pd.read_csv('train.csv', low_memory=False)

Cały preprocessing z poprzednich kamieni milowych sprowadziliśmy do pojedynczej funkcji.

In [ ]:
# Funkcje pomocnicze
def zamiana_typów_kolumn_na_numeryczne(df):
    df['Outstanding_Debt'] = Object_to_Numeric(df, 'Outstanding_Debt')
    df['Amount_invested_monthly'] = Object_to_Numeric(df, 'Amount_invested_monthly')
    df['Changed_Credit_Limit'] = Object_to_Numeric(df, 'Changed_Credit_Limit')
    df['Monthly_Balance'] = df['Monthly_Balance'].str.replace('_', '')
    df = df.astype({'Monthly_Balance': 'float'})
    df['Credit_History_Age'] = History_Age_2_months(df)
    df['Credit_History_Age'].loc[df.Credit_History_Age.isnull()] = df['Credit_History_Age'].loc[df.Credit_History_Age.isnull()].apply(
          lambda x: '-1') #Brak danych jest teraz -1 
    df['Credit_History_Age'] = df['Credit_History_Age'].apply(eval)
    df['Payment_Behaviour'] = np.where(df['Payment_Behaviour']== '!@9#%8', np.nan, df['Payment_Behaviour'])
    df['spent'] = np.where('Low' == df['Payment_Behaviour'].str[0:3], 0,1)
    #2 Month na numeric
    df['Month'] = df['Month'].apply(lambda mname: datetime.strptime(mname, '%B').month)

    #4 Age na numeric
    df['Age'] = df['Age'].convert_dtypes().apply(lambda x: x.replace("_", "")).astype(int)

    #7 Annual_Income na numeric
    df['Annual_Income'] = df['Annual_Income'].convert_dtypes().apply(lambda x: x.replace("_", "")).astype(float)

    #12 Num_of_Loan na numeric
    df['Num_of_Loan'] = df['Num_of_Loan'].convert_dtypes().apply(lambda x: x.replace("_", "")).astype(int)

    #15 Num_of_Delayed_Payment na numeric
    df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].convert_dtypes()
    df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].str.replace('_', '')
    df['Num_of_Delayed_Payment'] = pd.to_numeric(df['Num_of_Delayed_Payment'])

    df['payment_value'] = df['Payment_Behaviour'].str.extract(r'^(?:[^_]+_){2}([^_ ]+)')
    df['payment_value'] = df['payment_value'].replace('Small', 0)
    df['payment_value'] = df['payment_value'].replace('Medium', 1)
    df['payment_value'] = df['payment_value'].replace('Large', 2)

    df = df.drop('Payment_Behaviour', axis=1)

    # __ część z one_hot ___
    # Type_of_Loan
    df['Type_of_Loan'] = df['Type_of_Loan'].str.replace('and ', '')
    df.Type_of_Loan = df.Type_of_Loan.str.split(', ')
    df['Type_of_Loan'].loc[df.Type_of_Loan.isnull()] = df['Type_of_Loan'].loc[df.Type_of_Loan.isnull()].apply(lambda x: [])
    mlb = MultiLabelBinarizer(sparse_output=True)
    df = df.join(
        pd.DataFrame.sparse.from_spmatrix(
            mlb.fit_transform(df.pop('Type_of_Loan')),
            index=df.index,
            columns=mlb.classes_))
    
    # Credit_Mix i Occupation
    one_hot_encoded = pd.get_dummies(df['Credit_Mix'], prefix='Credit_Mix')
    df = pd.concat([df, one_hot_encoded], axis=1)
    one_hot_encoded = pd.get_dummies(df['Occupation'], prefix='Occupation')
    df = pd.concat([df, one_hot_encoded], axis=1)

    #Payment_of_Min_Amount
    grouped = df.groupby('Customer_ID', group_keys=False)
    df[['Payment_of_Min_Amount']] = grouped[['Payment_of_Min_Amount']].apply(lambda x: x.fillna(x.mode()))
    df['Payment_of_Min_Amount'].fillna(df['Payment_of_Min_Amount'].mode()[0])
    one_hot_encoded = pd.get_dummies(df['Payment_of_Min_Amount'], prefix='Payment_of_Min_Amount')
    df = pd.concat([df, one_hot_encoded], axis=1)
    
    df.drop(columns=['ID','SSN','Name'], inplace=True) #SSN to be removed
    df.drop(columns=['Payment_of_Min_Amount', 'Credit_Mix', 'Occupation', 'Payment_of_Min_Amount'], inplace=True)
    return df

In [ ]:
def History_Age_2_months(df):
      df['Credit_History_Age'] = df['Credit_History_Age'].str.replace(' Months', '')
      return df['Credit_History_Age'].str.replace('\D+', '* 12 +')

def Object_to_Numeric(df, s):
    return pd.to_numeric(df[s].str.replace('_', ''))

In [ ]:
def preprocess(df):

  def Zamiana_nan_na_medianę_modę(df):

    # Zamiana na NaN, a nie usunięcie rekordu
    df.Monthly_Balance = np.where(df.Monthly_Balance == min(df.Monthly_Balance), np.nan, df.Monthly_Balance)
    grouped = df.groupby('Customer_ID', group_keys=False)

    # Zamiana NaN na medianę dla danej grupy (Grupa = zbiór rekordów danego użytkownika, identyfikowany przez jego SSN)
    df[['Monthly_Inhand_Salary']] = grouped[['Monthly_Inhand_Salary']].apply(lambda x: x.fillna(x.median()))
    df[['Num_Bank_Accounts']] = grouped[['Num_Bank_Accounts']].apply(lambda x: x.fillna(x.median()))
    df[['Num_of_Delayed_Payment']] = grouped[['Num_of_Delayed_Payment']].apply(lambda x: x.fillna(x.median()))
    df[['Changed_Credit_Limit']] = grouped[['Changed_Credit_Limit']].apply(lambda x: x.fillna(x.median()))
    df[['Num_Credit_Inquiries']] = grouped[['Num_Credit_Inquiries']].apply(lambda x: x.fillna(x.median()))
    df[['Monthly_Balance']] = grouped[['Monthly_Balance']].apply(lambda x: x.fillna(x.median()))
    df[['Amount_invested_monthly']] = grouped[['Amount_invested_monthly']].apply(lambda x: x.fillna(x.median()))
    df[['payment_value']] = grouped[['payment_value']].apply(lambda x: x.fillna(x.mode()))

    df['Num_Bank_Accounts'].fillna(df['Num_Bank_Accounts'].median(), inplace=True) 
    df['Num_of_Delayed_Payment'].fillna(df['Num_of_Delayed_Payment'].median(), inplace=True) 
    df['Num_Bank_Accounts'].fillna(df['Num_Bank_Accounts'].median(), inplace=True) 
    df['Changed_Credit_Limit'].fillna(df['Changed_Credit_Limit'].median(), inplace=True) 
    df['Num_Credit_Inquiries'].fillna(df['Num_Credit_Inquiries'].median(), inplace=True) 
    df['Monthly_Inhand_Salary'].fillna(df['Monthly_Inhand_Salary'].median(), inplace=True) 
    df['Monthly_Balance'].fillna(df['Monthly_Balance'].median(), inplace=True) 
    df['payment_value'].fillna(df['payment_value'].mode()[0], inplace=True) 
    df['Amount_invested_monthly'].fillna(df['Amount_invested_monthly'].median(), inplace=True)
    return df
  
  df = zamiana_typów_kolumn_na_numeryczne(df)
  df = Zamiana_nan_na_medianę_modę(df)
  return df

In [ ]:
X_TRAIN, X_valid = train_test_split(df, test_size=0.30, random_state=42)
y_test = X_test['Credit_Score']

In [ ]:
target = X_TRAIN['Credit_Score']
X_train, X_test=train_test_split(
    X_TRAIN, # X
    test_size=0.30, random_state=42)

In [ ]:
X_train = preprocess(X_train)

<ipython-input-84-c974350e5715>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  return df['Credit_History_Age'].str.replace('\D+', '* 12 +')
<ipython-input-84-c974350e5715>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Credit_History_Age'].loc[df.Credit_History_Age.isnull()] = df['Credit_History_Age'].loc[df.Credit_History_Age.isnull()].apply(
<ipython-input-84-c974350e5715>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Type_of_Loan'].loc[df.Type_of_Loan.isnull()] = df['Type_of_Loan'].loc[df.Type_of_Loan.isnull()].apply(lambda x: [])


In [ ]:
X_test = zamiana_typów_kolumn_na_numeryczne(X_test)

<ipython-input-13-6824ee280100>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  return df['Credit_History_Age'].str.replace('\D+', '* 12 +')
<ipython-input-8-f0c825b1f133>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Credit_History_Age'].loc[df.Credit_History_Age.isnull()] = df['Credit_History_Age'].loc[df.Credit_History_Age.isnull()].apply(
<ipython-input-8-f0c825b1f133>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Type_of_Loan'].loc[df.Type_of_Loan.isnull()] = df['Type_of_Loan'].loc[df.Type_of_Loan.isnull()].apply(lambda x: [])


In [ ]:
grouped = X_train.groupby('Customer_ID', group_keys=False)
grouped2 = X_test.groupby('Customer_ID', group_keys=False)

col_names = ['Monthly_Inhand_Salary','Num_Bank_Accounts','Num_of_Delayed_Payment','Changed_Credit_Limit','Num_Credit_Inquiries','Monthly_Balance',
             'Amount_invested_monthly']

for col_name in col_names:
  new_table = np.array([]) 
  for key, item in grouped2:
    if key in grouped.groups.keys():
      if not math.isnan(grouped[[col_name]].get_group(key).median()[0]):
        item[col_name].fillna(grouped[[col_name]].get_group(key).median()[0], inplace=True)
        item1 = item.copy()
        if new_table.shape[0]==0:
            new_table = item1
        else:
            new_table = pd.concat([new_table,item1])
      else:
        item.fillna(X_train[col_name].median(), inplace=True)
        item1 = item.copy()
        if new_table.shape[0]==0:
            new_table = item1
        else:
            new_table = pd.concat([new_table,item1])
    else:
      item.fillna(X_train[col_name].median(), inplace=True)
      item1 = item.copy()
      if new_table.shape[0]==0:
          new_table = item1
      else:
          new_table = pd.concat([new_table,item1])
  temp = new_table.copy()
  X_train[col_name] = temp[col_name]


new_table = np.array([]) 
for key, item in grouped2:
  if key in grouped.groups.keys():
    if not math.isnan(grouped[['payment_value']].get_group(key).mode()[0]):
      item['payment_value'].fillna(grouped[['payment_value']].get_group(key).mode()[0], inplace=True)
      item1 = item.copy()
      if new_table.shape[0]==0:
          new_table = item1
      else:
          new_table = pd.concat([new_table,item1])
    else:
      item.fillna(X_train['payment_value'].mode(), inplace=True)
      item1 = item.copy()
      if new_table.shape[0]==0:
          new_table = item1
      else:
          new_table = pd.concat([new_table,item1])
  else:
    item.fillna(X_train['payment_value'].mode(), inplace=True)
    item1 = item.copy()
    if new_table.shape[0]==0:
        new_table = item1
    else:
        new_table = pd.concat([new_table,item1])
X_train['payment_value'] = temp['payment_value']

In [ ]:
y_test = X_test['Credit_Score']
X_test.drop(columns=['Credit_Score', 'Customer_ID'], inplace=True)

y_train = X_train['Credit_Score']
X_train.drop(columns=['Credit_Score', 'Customer_ID'], inplace=True)

# Minmax Scaler i Outliery

Każda kolumna traktowana oddzielnie, jednak poziom odcięcia zbioru testowego bezpośrednio przenoszona jest ze zbioru treningowego, podobnie jak wartości minimalne/maksymalne przy minmax scalerze

In [ ]:
# Definiujemy nasz własny min_max scaler, by wartości min/max pochodziły ze
# zbioru treningowego.
def scale_mm(x_train, x_test):
  x_train_std = (x_train - x_train.min(axis=0)) / (x_train.max(axis=0) - x_train.min(axis=0))
  x_test_std = (x_test - x_train.min(axis=0)) / (x_train.max(axis=0) - x_train.min(axis=0))
  return x_train_std, x_test_std

In [ ]:
upper_lim = X_train.Annual_Income.quantile(.99)

X_train.Annual_Income = np.where(X_train.Annual_Income > upper_lim, upper_lim, X_train.Annual_Income)
X_test.Annual_Income = np.where(X_test.Annual_Income > upper_lim, upper_lim, X_test.Annual_Income)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [ ]:
X_train.Annual_Income, X_test.Annual_Income = scale_mm(X_train.Annual_Income, X_test.Annual_Income)

In [ ]:
upper_lim = X_train.Amount_invested_monthly.quantile(.98)

X_train.Amount_invested_monthly = np.where(X_train.Amount_invested_monthly > upper_lim, upper_lim, X_train.Amount_invested_monthly)
X_test.Amount_invested_monthly = np.where(X_test.Amount_invested_monthly > upper_lim, upper_lim, X_test.Amount_invested_monthly)

X_train.Amount_invested_monthly = np.log1p(X_train.Amount_invested_monthly)
X_test.Amount_invested_monthly = np.log1p(X_test.Amount_invested_monthly)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [ ]:
X_train.Amount_invested_monthly, X_test.Amount_invested_monthly = scale_mm(X_train.Amount_invested_monthly, X_test.Amount_invested_monthly)

In [ ]:
upper_lim = X_train.Total_EMI_per_month.quantile(.96)

X_train.Total_EMI_per_month = np.where(X_train.Total_EMI_per_month > upper_lim, upper_lim, X_train.Total_EMI_per_month)
X_test.Total_EMI_per_month = np.where(X_test.Total_EMI_per_month > upper_lim, upper_lim, X_test.Total_EMI_per_month)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

X_train.Total_EMI_per_month = np.log1p(X_train.Total_EMI_per_month)
X_test.Total_EMI_per_month = np.log1p(X_test.Total_EMI_per_month)

In [ ]:
X_train.Credit_Utilization_Ratio, X_test.Credit_Utilization_Ratio = scale_mm(X_train.Credit_Utilization_Ratio, X_test.Credit_Utilization_Ratio)

In [ ]:
X_train.Outstanding_Debt, X_test.Outstanding_Debt = scale_mm(X_train.Outstanding_Debt, X_test.Outstanding_Debt)

In [ ]:
upper_lim = X_train.Num_Credit_Inquiries.quantile(.96)

X_train.Num_Credit_Inquiries = np.where(X_train.Num_Credit_Inquiries > upper_lim, upper_lim, X_train.Num_Credit_Inquiries)
X_test.Num_Credit_Inquiries = np.where(X_test.Num_Credit_Inquiries > upper_lim, upper_lim, X_test.Num_Credit_Inquiries)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [ ]:
X_train.Changed_Credit_Limit, X_test.Changed_Credit_Limit = scale_mm(X_train.Changed_Credit_Limit, X_test.Changed_Credit_Limit)

In [ ]:
upper_lim = X_train.Age.quantile(.98)
lower_lim = X_train.Age.quantile(.05)

X_train.Age = np.where(X_train.Age > upper_lim, upper_lim, X_train.Age)
X_test.Age = np.where(X_test.Age > upper_lim, upper_lim, X_test.Age)

X_train.Age = np.where(X_train.Age < lower_lim, lower_lim, X_train.Age)
X_test.Age = np.where(X_test.Age < lower_lim, lower_lim, X_test.Age)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [ ]:
upper_lim = X_train.Num_Bank_Accounts.quantile(.98)
lower_lim = X_train.Num_Bank_Accounts.quantile(.00)

X_train.Num_Bank_Accounts = np.where(X_train.Num_Bank_Accounts > upper_lim, upper_lim, X_train.Num_Bank_Accounts)
X_test.Num_Bank_Accounts = np.where(X_test.Num_Bank_Accounts > upper_lim, upper_lim, X_test.Num_Bank_Accounts)

X_train.Num_Bank_Accounts = np.where(X_train.Num_Bank_Accounts < lower_lim, lower_lim, X_train.Num_Bank_Accounts)
X_test.Num_Bank_Accounts = np.where(X_test.Num_Bank_Accounts < lower_lim, lower_lim, X_test.Num_Bank_Accounts)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [ ]:
upper_lim = X_train.Num_Credit_Card.quantile(.97)
lower_lim = X_train.Num_Credit_Card.quantile(.00)

X_train.Num_Credit_Card = np.where(X_train.Num_Credit_Card > upper_lim, upper_lim, X_train.Num_Credit_Card)
X_test.Num_Credit_Card = np.where(X_test.Num_Credit_Card > upper_lim, upper_lim, X_test.Num_Credit_Card)

X_train.Num_Credit_Card = np.where(X_train.Num_Credit_Card < lower_lim, lower_lim, X_train.Num_Credit_Card)
X_test.Num_Credit_Card = np.where(X_test.Num_Credit_Card < lower_lim, lower_lim, X_test.Num_Credit_Card)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [ ]:
upper_lim = X_train.Interest_Rate.quantile(.97)

X_train.Interest_Rate = np.where(X_train.Interest_Rate > upper_lim, upper_lim, X_train.Interest_Rate)
X_test.Interest_Rate = np.where(X_test.Interest_Rate > upper_lim, upper_lim, X_test.Interest_Rate)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [ ]:
upper_lim = X_train.Num_of_Loan.quantile(.99)
lower_lim = X_train.Num_of_Loan.quantile(.04)
X_train.Num_of_Loan = np.where(X_train.Num_of_Loan > upper_lim, upper_lim, X_train.Num_of_Loan)
X_test.Num_of_Loan = np.where(X_test.Num_of_Loan > upper_lim, upper_lim, X_test.Num_of_Loan)

X_train.Num_of_Loan = np.where(X_train.Num_of_Loan < lower_lim, lower_lim, X_train.Num_of_Loan)
X_test.Num_of_Loan = np.where(X_test.Num_of_Loan < lower_lim, lower_lim, X_test.Num_of_Loan)

X_train =X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [ ]:
upper_lim = X_train.Delay_from_due_date.quantile(.99)

X_train.Delay_from_due_date = np.where(X_train.Delay_from_due_date > upper_lim, upper_lim, X_train.Delay_from_due_date)
X_test.Delay_from_due_date = np.where(X_test.Delay_from_due_date > upper_lim, upper_lim, X_test.Delay_from_due_date)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [ ]:
upper_lim = X_train.Num_of_Delayed_Payment.quantile(.99)
lower_lim = X_train.Num_of_Delayed_Payment.quantile(.01)

X_train.Num_of_Delayed_Payment = np.where(X_train.Num_of_Delayed_Payment > upper_lim, upper_lim, X_train.Num_of_Delayed_Payment)
X_test.Num_of_Delayed_Payment = np.where(X_test.Num_of_Delayed_Payment > upper_lim, upper_lim, X_test.Num_of_Delayed_Payment)

X_train.Num_of_Delayed_Payment = np.where(X_train.Num_of_Delayed_Payment < lower_lim, upper_lim, X_train.Num_of_Delayed_Payment)
X_test.Num_of_Delayed_Payment = np.where(X_test.Num_of_Delayed_Payment < lower_lim, upper_lim, X_test.Num_of_Delayed_Payment)

In [ ]:
# Usunięcie kolumn na podstawie korelacji spearmana (>90)
X_train = X_train.drop(columns=['Monthly_Inhand_Salary', 'Month'])
X_test = X_test.drop(columns=['Monthly_Inhand_Salary', 'Month'])

In [ ]:
y_test = y_test.replace('Good', 2)
y_test = y_test.replace('Standard', 1)
y_test = y_test.replace('Poor', 0)

y_train = y_train.replace('Good', 2)
y_train = y_train.replace('Standard', 1)
y_train = y_train.replace('Poor', 0)

# Model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings('ignore')
import numpy as np

In [ ]:
def get_models():
    
    models = dict()
    models['cart'] = DecisionTreeClassifier()
    models['knn'] = KNeighborsClassifier()

    models['rf'] = RandomForestClassifier(n_estimators=1000, # Ilość słabych estymatorów
                                      max_depth=2, # Maksymalna wysokość drzewa w słabym estymatorze
                                      min_samples_split = 2, # Minimalna ilość obserwacji wymagana do podziału węzła
                                      max_features = 3, # Maksymalna ilość zmiennych brana pod uwagę przy podziale węzła
                                      random_state=0,
                                      n_jobs = -1)
    
    models['aboost'] = AdaBoostClassifier(random_state=1)

    models['gboost'] = GradientBoostingClassifier(random_state=1,
                                      learning_rate=0.01)
    
    models['xgb'] = XGBClassifier(random_state=1,
                        learning_rate=0.01, # Szybkość "uczenia" się
                        booster='gbtree', # Jaki model wykorzystujemy (drzewo - gbtree, liniowe - gblinear)
                        max_depth=4 # Maksymalna głębokość drzewa 
                        )
    
    models['svm'] = SVC()

    models['bayes'] = GaussianNB()

    return models

In [ ]:
def evaluate_model(model, X, y):
    
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

    return scores

In [ ]:
# modele do evaluacji
models = get_models()
# ocena modeli
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X_train, y_train)
    results.append(scores)
    names.append(name)
    print('%8s mean-%.3f std-%.3f' % (name, np.mean(scores), np.std(scores)))